In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

import hervpd as hp
import csvUtils as csvu
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)

from datetime import datetime, timedelta

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [2]:
    verbose = True    
    user = 0
    
    RAW_DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/Raw/%d"%user
    PRE_DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/PreProcessed/%d"%user
    
    # duration (in seconds) to be cropped from the beginning of each second to account for stabilization and user adjustment
    crop = 90
    
    # duration (in seconds) of each fragment to be sent to analysis
    duration = 300
    
    # if any fragment has more than 'threshold' consecutive seconds with no beats, it will be discarded
    threshold = 3

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [3]:
sessions = paf.parseActivityFiles(dirname=RAW_DATA_PATH)

reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171014.csv ... 
orphan stop in: ['2017-10-14 2:14:21', 'stop', '', '', '']
orphan start in: ['2017-10-14 11:58:31', 'start', 'focused-active', 'lying down', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171102.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171016.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171006.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171104.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171029.csv ... 
orphan stop in: ['2017-10-29 15:27:59', 'stop', '', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171028.csv ... 
orphan start in: ['2017-10-28 11:48:07', 'start', 'sleep', 'posture']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171013.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171003.csv ... 
orphan start in: ['2017-10-03 22:56:19', 'start', 'eat', 'sitting']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/

## 2 - Generate fragments from sessions

### 1.2 Extract session intervals

Retrieves from the heartbeat files all the intervals contained in each session's duration and adds them to the sessions objects (in memory)

In [28]:
frags = fragment_sessions(sessions, duration, crop)

print (sessions[0])

df = pd.DataFrame(frags)
df.head(10)

122 valid sessions out of 157 total (at least one full fragment of 300 seconds after discarding first 90 seconds)
{'start': datetime.datetime(2017, 10, 14, 2, 14, 21), 'activity': 'focused-active', 'posture': 'sitting', 'notes': '', 'stop': datetime.datetime(2017, 10, 14, 2, 47, 10), 'duration': 1969}


,activity,order,rr,sess,start,stop
0,focused-active,0,"[{'date': 2017-10-14 02:15:52, 'interval': 745...",0,2017-10-14 02:15:51,2017-10-14 02:20:51
1,focused-active,1,"[{'date': 2017-10-14 02:20:52, 'interval': 764...",0,2017-10-14 02:20:51,2017-10-14 02:25:51
2,focused-active,2,"[{'date': 2017-10-14 02:25:52, 'interval': 831...",0,2017-10-14 02:25:51,2017-10-14 02:30:51
3,focused-active,3,"[{'date': 2017-10-14 02:30:52, 'interval': 830...",0,2017-10-14 02:30:51,2017-10-14 02:35:51
4,focused-active,4,"[{'date': 2017-10-14 02:35:52, 'interval': 829...",0,2017-10-14 02:35:51,2017-10-14 02:40:51
5,focused-active,5,"[{'date': 2017-10-14 02:40:52, 'interval': 671...",0,2017-10-14 02:40:51,2017-10-14 02:45:51
6,focused-active,0,"[{'date': 2017-10-14 02:59:03, 'interval': 798...",1,2017-10-14 02:59:02,2017-10-14 03:04:02
7,focused-active,1,"[{'date': 2017-10-14 03:04:03, 'interval': 823...",1,2017-10-14 03:04:02,2017-10-14 03:09:02
8,focused-active,0,"[{'date': 2017-10-14 03:12:51, 'interval': 107...",2,2017-10-14 03:12:50,2017-10-14 03:17:50
9,focused-active,1,"[{'date': 2017-10-14 03:17:51, 'interval': 769...",2,2017-10-14 03:17:50,2017-10-14 03:22:50
